In [113]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("../")
sys.path.append("../math_tricks/")
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime

# multiprocessing
import multiprocessing
from functools import partial
# filtro distancia
from scipy.spatial import distance

In [114]:
timenow_bueno = datetime.datetime.now()

timenow = datetime.datetime.now()

# lectura de archivo
file1 = '../pdbs/1xxa_clean.pdb'  # sys.argv[1]
file2 = '../pdbs/1tig_clean.pdb'  # sys.argv[2]

# numero de cliques, preguntar en el software para generalizarlo INPUT...
number_elements_clique = 3

# se define la estructura
pdb1 = rpt.PdbStruct(file1)
pdb2 = rpt.PdbStruct(file2)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)
pdb2.AddPdbData("%s" % file2)

# Se calculan sus vecinos mas cercanos
pdb1.GetNeighbors()
pdb2.GetNeighbors()

# se obtienen los residuos que perteneces a la cadena de interes por default chain = 'A'
pdb11 = pdb1.GetResChain()
pdb22 = pdb2.GetResChain()

# Siempre la proteina 1 es el que se rota y traslada para embonar en la proteina 2
if len(pdb22) < len(pdb11):

    import copy
    pdb1_temp = copy.copy(pdb1)
    pdb2_temp = copy.copy(pdb2)

    pdb11_temp = copy.copy(pdb11)
    pdb22_temp = copy.copy(pdb22)

    pdb1 = pdb2_temp
    pdb2 = pdb1_temp

    pdb11 = pdb22_temp
    pdb22 = pdb11_temp

    del [pdb1_temp]
    del [pdb2_temp]
    del [pdb11_temp]
    del [pdb22_temp]

    print("Intercambio de nombre ya que la proteina 1 es mas grande que la 2")
    print(pdb1.name, len(pdb11))
    print(pdb2.name, len(pdb22))
    print("No te preocupes ya quedo :)")


pdb1.Set_SS()
pdb2.Set_SS()

ss1 = fc.create_ss_table(pdb11)
ss2 = fc.create_ss_table(pdb22)


def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    lista_residuos = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = [ [pdist(np.array([v, av]), metric='euclidean').item() for av in coord] for v in coord]

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=lista_residuos, columns=lista_residuos, data=distancias)
    return(df_da, lista_residuos)


# devuelve tabla e indices de el dataframe de distancias entre atomos de la misma proteina con dth < 10A
df_distancias1, lista_residuos_1 = get_df_distancias(pdb11)
df_distancias2, lista_residuos2 = get_df_distancias(pdb22)

# se generan cliques, te devuleve dataframe con cliques de k(numero_de_cliques) y la lista de cliques maximales
df_cliques1, cliques1 = fc.gen_3_cliques(df_distancias1,  dth=10, k=number_elements_clique)  # file1[:-4] pal gexf
print('**'*50)
df_cliques2, cliques2 = fc.gen_3_cliques(df_distancias2,  dth=10, k=number_elements_clique)  # file2[:-4]
print('**'*50)

# REVISAR DONDE GUARDA EL GEXF Y COLOCARLO EN LA CARPETA Grafos

# se agrega filtro de residuos que pertenecen a cliques de 7 elementos
mc1_7 = cliques1[cliques1.numero_elementos == 7].drop('numero_elementos', 1)
mc2_7 = cliques2[cliques2.numero_elementos == 7].drop('numero_elementos', 1)

residuos_unicos_1 = []
for i in [list(mc1_7[i].unique()) for i in mc1_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_1.append(int(j))

residuos_unicos_2 = []
for i in [list(mc2_7[i].unique()) for i in mc2_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_2.append(int(j))

residuos_unicos_1 = set(residuos_unicos_1)
residuos_unicos_2 = set(residuos_unicos_2)


# funcion para obtener las propiedades del residuo para los cliques agrupados
def get_df_ca(list_of_residues):
    """Genera dataframe con la informacion necesaria para las siguientes funciones
    FALTA DOCUMENTAR ESTA COSA!!!!"""
    #crear df_ca
    atom_number = []
    atom_name = []
    residue_name = []
    residue_number = []
    coord = []
    res_ngb = []
    for res in list_of_residues:
        for atom in res.atoms:
            atom_number.append(atom.atom_number)
            atom_name.append(atom.name)
            residue_name.append(res.resn)
            residue_number.append(res.resi)
            coord.append(atom.coord)
            res_ngb.append(res.ngb)

    df_atoms = pd.DataFrame(columns=['atom_number', 'atom_name', 'residue_name',
                                   'residue_number', 'vector', 'ngb'])
    df_atoms.atom_number = atom_number
    df_atoms.atom_name = atom_name
    df_atoms.residue_name = residue_name
    df_atoms.residue_number = residue_number
    df_atoms.vector = coord
    df_atoms.ngb = res_ngb

    return(df_atoms)


# CREAR DF_atomos_CA #
df_atoms1 = get_df_ca(pdb11)
df_atoms2 = get_df_ca(pdb22)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
****************************************************************************************************
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102
****************************************************************************************************


In [83]:
96,97,91
[154,156,119]
parejas = [(96,154),(97,157),(91,119)]

In [84]:
vecs1 = np.array([vecs for vecs in df_atoms1.vector.values])
vecs2 = np.array([vecs for vecs in df_atoms2.vector.values])

# baricentro
bari_1 = vecs1.mean(0)
bari_2 = vecs2.mean(0)

# vectores centricos
vecs_center_1 = vecs1 - bari_1
vecs_center_2 = vecs2 - bari_2

In [85]:
def R_ij(i, j, parejas=parejas):
    valor = sum([vecs_center_1[pos_res[0], i] * vecs_center_2[pos_res[1], j] for pos_res in parejas])
    return valor

def matrix_R():
    """cliques a comparar: i,j
    desde aqui se itera sobre cada i y hay que variar los vectores
    coordenada
    Regresa la matriz gigante (matriz simetrica del articulo)"""
    # primer renglon
    R11R22R33 = (R_ij(0, 0) + R_ij(1, 1) + R_ij(2, 2))
    R23_R32 = (R_ij(1, 2) - R_ij(2, 1))
    R31_R13 = (R_ij(2, 0) - R_ij(0, 2))
    R12_R21 = (R_ij(0, 1) - R_ij(1, 0))
    # segundo renglon
    R11_R22_R33 = (R_ij(0, 0) - R_ij(1, 1) - R_ij(2, 2))
    R12R21 = (R_ij(0, 1) + R_ij(1, 0))
    R13R31 = (R_ij(0, 2) + R_ij(2, 0))
    # tercer renglon
    _R11R22_R33 = (-R_ij(0, 0) + R_ij(1, 1) - R_ij(2, 2))
    R23R32 = (R_ij(1, 2) + R_ij(2, 1))
    # cuarto renglon
    _R11_R22R33 = (-R_ij(0, 0) - R_ij(1, 1) + R_ij(2, 2))

    matriz_R = [
        [R11R22R33, R23_R32, R31_R13, R12_R21],
        [R23_R32, R11_R22_R33, R12R21, R13R31],
        [R31_R13, R12R21, _R11R22_R33, R23R32],
        [R12_R21, R13R31, R23R32, _R11_R22R33]
    ]
    return (matriz_R)

def rotation_matrix(matriz_R):
    """utilizando la funcion giant_matrix, fijando los valores de i,j
    se calcula la matriz de rotacion con los eigenvectores y eigenvalores
    arroja una matriz de rotacion que depende de la matriz gigante
    """
    eignvalues, eigenvectors = np.linalg.eig(matriz_R)
    q = eigenvectors[:, np.argmax(eignvalues)]
    q0, q1, q2, q3 = q[0], q[1], q[2], q[3]
    # matriz de rotacion con eigenvectores
    matriz_rotacion = np.array([
        [(q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2), 2 * (q1 * q2 - q0 * q3), 2 * (q1 * q3 + q0 * q2)],
        [2 * (q1 * q2 + q0 * q3), (q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2), 2 * (q2 * q3 - q0 * q1)],
        [2 * (q1 * q3 - q0 * q2), 2 * (q2 * q3 + q0 * q1), (q0 ** 2 - q1 ** 2 - q2 ** 2 + q3 ** 2)]
    ], dtype=np.float64)
    return (matriz_rotacion)

def rotation_vectors(vector_gorro, matriz_rotacion):
    """obtencion de vector rotado,
    utilizando la matriz de rotacion
    y los vectores gorro a rotar y trasladar"""

    coord_rotado = [np.matmul(
        matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    return (coord_rotado)

def rmsd_between_cliques(clique_trasladado_rotado, atom_to_compare):
    """Calculo de rmsd entre cliques tomando el atomo rotado y trasladado
    y el atomo a comparar, por el momento solo imprime el resultado"""
    # primer RMSD entre atomos
    p12 = np.sum((np.array(
        atom_to_compare, dtype=np.float64) - clique_trasladado_rotado) ** 2, 1)
    rmsd_i = lambda i: np.sqrt(i) / 3
    rmsd_final = rmsd_i(p12).mean()

    return (rmsd_final)

In [98]:
matriz_rotacion

array([[ 0.2272509 , -0.82132285, -0.52324546],
       [ 0.48833645,  0.56097525, -0.66845664],
       [ 0.84254646, -0.10361246,  0.52856401]])

In [100]:
mat_r = np.array([[0.9914070514132912, 0.07582408401406437, 0.1065962789755417], [0.1298933992885212, -0.47415967110042934, -0.8708044046301128], [-0.015484289756918512, 0.8771677801796874, -0.4799342894452979]])

In [101]:
# matriz_R = matrix_R()
# matriz_rotacion = fc.rotation_matrix(matriz_R)
vector_rotado = fc.rotation_vectors(vecs_center_1, mat_r)
vector_rotado_trasladado_a_clique2 = vector_rotado + np.array(bari_2)
protein_trasladado_rotado = vector_rotado_trasladado_a_clique2
protein_to_compare = vecs2

In [102]:
df_cliques1.iloc[12]

0    96
1    97
2    91
Name: 12, dtype: int64

In [103]:
(12, 372)

(12, 372)

In [104]:
rmsd_between_cliques(protein_trasladado_rotado[[96,97,91]],protein_to_compare[[154,156,119]])

7.157964105605321

In [105]:
vecs1[:3]

array([[28.39 , 57.093, 26.056],
       [28.952, 56.593, 24.813],
       [28.021, 57.009, 23.678]])

In [106]:
protein_trasladado_rotado[[96,97,91]]

array([[ 5.22313615, 10.66319764, 22.27097388],
       [ 5.03930159, 12.16014327, 22.08940125],
       [ 4.45803217,  8.08933989, 18.14701229]])

In [107]:
protein_to_compare[[154,156,119]]

array([[13.549,  9.51 ,  1.501],
       [14.086, 10.457,  3.654],
       [11.814,  6.76 , -1.915]])

In [108]:
protein_trasladado_rotado[[96,97,91]] - protein_to_compare[[154,156,119]]

array([[-8.32586385,  1.15319764, 20.76997388],
       [-9.04669841,  1.70314327, 18.43540125],
       [-7.35596783,  1.32933989, 20.06201229]])

In [109]:
(protein_trasladado_rotado[[96,97,91]] - protein_to_compare[[154,156,119]]) ** 2

array([[ 69.32000885,   1.3298648 , 431.3918151 ],
       [ 81.84275215,   2.90069701, 339.86401927],
       [ 54.11026277,   1.76714455, 402.48433724]])

In [110]:
np.sum((protein_trasladado_rotado[[96,97,91]] - protein_to_compare[[154,156,119]]) ** 2,1)

array([502.04168875, 424.60746843, 458.36174456])

In [111]:
np.sqrt(np.sum((protein_trasladado_rotado[[96,97,91]] - protein_to_compare[[154,156,119]]) ** 2,1))/3

array([7.46876227, 6.86866855, 7.1364615 ])

In [112]:
np.mean(np.sqrt(np.sum((protein_trasladado_rotado[[96,97,91]] - protein_to_compare[[154,156,119]]) ** 2,1))/3)

7.157964105605321